#Задание 1.
Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы
 
#Задание 2.
Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [30]:
!wget https://www.dropbox.com/s/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz

--2022-11-09 18:55:12--  https://www.dropbox.com/s/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz [following]
--2022-11-09 18:55:12--  https://www.dropbox.com/s/raw/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucae4814d6843a11e8813b0b9197.dl.dropboxusercontent.com/cd/0/inline/BwdN44rprBL7BypAVmqU_RcuJlloMuvlqb9XZAhD7X8MOYqT9wJUfA-SSTD7HD28UDDNbOL8A6lLymeXRpQzi0bvo5Nj7mfAtx6dUHUJ0xryot3_frKHFF0sxPAjwig61hKMVu5G-OuqZ1C1P7Y6HgF4IsO_iKnBFYx18xiv1HjL9Q/file# [following]
--2022-11-09 18:55:13--  https://ucae4814d6843a11e8813b0b9197.dl.dropboxusercontent.com/cd/0/inline/BwdN44rprBL7BypAVmqU_RcuJlloMuvlqb9XZAhD7X8MOYqT9w

In [31]:
!tar -xzvf "/content/gazeta_jsonl_v2.tar.gz"

gazeta_train.jsonl
gazeta_val.jsonl
gazeta_test.jsonl


In [32]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.8 MB/s 


In [58]:
%matplotlib inline
import pprint
import json
import numpy as np
import pandas as pd
import re
import pymorphy2
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger
nltk.download('punkt')
from tqdm.notebook import tqdm
from corus import load_ne5

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [34]:
records = []
with open("/content/gazeta_train.jsonl", "r") as r:
    for line in r:
        records.append(json.loads(line))

In [35]:
# records

In [36]:
text_corpus = []
for i in range(len(records)):
  fg = records[i].get('url')
  result = re.sub('/', '', re.findall(r'/\w+', fg)[1])
  if result == 'auto':
    text_corpus.append(records[i].get('summary'))

text_corpus[0]

'С 2011 года правительство отменяет самый раздражающий граждан налог – транспортный. Но поборы автомобилистов не прекратятся – налоги завуалируют в бензиновые акцизы и платные дороги, а цены на товары подскочат. Зато теперь собираемые деньги обещают пустить только на строительство и содержание дорог.'

In [37]:
len(text_corpus)

2995

In [38]:
words_regex = re.compile('\w+')

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')

morph = pymorphy2.MorphAnalyzer()

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = set(nltk.corpus.stopwords.words("russian"))


def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


In [39]:
preprocessed_text = list(tqdm(map(preprocess, text_corpus), total=len(text_corpus)))

  0%|          | 0/2995 [00:00<?, ?it/s]

In [40]:
words = list(tqdm(map(find_words, text_corpus), total=len(text_corpus)))

  0%|          | 0/2995 [00:00<?, ?it/s]

In [41]:
train_data = words[1:int(len(words) * 0.9)]
test_data = words[int(len(words) * 0.9):]
test_sent = words[0]

In [42]:
train_data = [nltk.pos_tag(i, lang = 'rus') for i in train_data]
test_data = [nltk.pos_tag(i, lang = 'rus') for i in test_data]
test_sent = nltk.pos_tag(test_sent, lang = 'rus')

In [43]:
unigram_tagger = UnigramTagger(train_data)
display(unigram_tagger.tag(test_sent))
unigram_tagger.evaluate(test_data)

[(('года', 'S'), None),
 (('правительство', 'S'), None),
 (('отменяет', 'V'), None),
 (('самый', 'A-PRO=m'), None),
 (('раздражающий', 'V'), None),
 (('граждан', 'S'), None),
 (('налог', 'S'), None),
 (('транспортный', 'A=m'), None),
 (('поборы', 'S'), None),
 (('автомобилистов', 'S'), None),
 (('прекратятся', 'V'), None),
 (('налоги', 'S'), None),
 (('завуалируют', 'V'), None),
 (('бензиновые', 'A=pl'), None),
 (('акцизы', 'S'), None),
 (('платные', 'A=pl'), None),
 (('дороги', 'S'), None),
 (('цены', 'S'), None),
 (('товары', 'S'), None),
 (('подскочат', 'V'), None),
 (('зато', 'CONJ'), None),
 (('теперь', 'ADV-PRO'), None),
 (('собираемые', 'V'), None),
 (('деньги', 'S'), None),
 (('обещают', 'V'), None),
 (('пустить', 'V'), None),
 (('только', 'PART'), None),
 (('строительство', 'S'), None),
 (('содержание', 'S'), None),
 (('дорог', 'S'), None)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0.8033448188923439

In [44]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent))
bigram_tagger.evaluate(test_data)

[(('года', 'S'), None),
 (('правительство', 'S'), None),
 (('отменяет', 'V'), None),
 (('самый', 'A-PRO=m'), None),
 (('раздражающий', 'V'), None),
 (('граждан', 'S'), None),
 (('налог', 'S'), None),
 (('транспортный', 'A=m'), None),
 (('поборы', 'S'), None),
 (('автомобилистов', 'S'), None),
 (('прекратятся', 'V'), None),
 (('налоги', 'S'), None),
 (('завуалируют', 'V'), None),
 (('бензиновые', 'A=pl'), None),
 (('акцизы', 'S'), None),
 (('платные', 'A=pl'), None),
 (('дороги', 'S'), None),
 (('цены', 'S'), None),
 (('товары', 'S'), None),
 (('подскочат', 'V'), None),
 (('зато', 'CONJ'), None),
 (('теперь', 'ADV-PRO'), None),
 (('собираемые', 'V'), None),
 (('деньги', 'S'), None),
 (('обещают', 'V'), None),
 (('пустить', 'V'), None),
 (('только', 'PART'), None),
 (('строительство', 'S'), None),
 (('содержание', 'S'), None),
 (('дорог', 'S'), None)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0.8042692663249013

In [45]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent))
trigram_tagger.evaluate(test_data)

[(('года', 'S'), None),
 (('правительство', 'S'), None),
 (('отменяет', 'V'), None),
 (('самый', 'A-PRO=m'), None),
 (('раздражающий', 'V'), None),
 (('граждан', 'S'), None),
 (('налог', 'S'), None),
 (('транспортный', 'A=m'), None),
 (('поборы', 'S'), None),
 (('автомобилистов', 'S'), None),
 (('прекратятся', 'V'), None),
 (('налоги', 'S'), None),
 (('завуалируют', 'V'), None),
 (('бензиновые', 'A=pl'), None),
 (('акцизы', 'S'), None),
 (('платные', 'A=pl'), None),
 (('дороги', 'S'), None),
 (('цены', 'S'), None),
 (('товары', 'S'), None),
 (('подскочат', 'V'), None),
 (('зато', 'CONJ'), None),
 (('теперь', 'ADV-PRO'), None),
 (('собираемые', 'V'), None),
 (('деньги', 'S'), None),
 (('обещают', 'V'), None),
 (('пустить', 'V'), None),
 (('только', 'PART'), None),
 (('строительство', 'S'), None),
 (('содержание', 'S'), None),
 (('дорог', 'S'), None)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0.8038490629464661

##В отличии от английского языка для русского nltk работает хуже и на триграме и комбинации тегеров не дает прироста качества

#Написать свой теггер как на занятии

In [48]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-11-09 19:10:50--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.54MB/s    in 1.2s    

2022-11-09 19:10:52 (1.54 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [50]:
!unzip "/content/collection5.zip"

Archive:  /content/collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/01

In [56]:
dir = '/content/Collection5/'

records = load_ne5(dir)


t_df = pd.DataFrame(columns = ['index', 'type', 'text'])
for item in records:
    for i in item.spans:
        if i.type == 'PER':
            t_df.loc[len(t_df)] = [i.index, i.type, i.text] 

names = list(set(t_df['text']))

In [59]:
full_n = []
for s in tqdm(names):
    full_n.append(s)
    full_n.append(nltk.word_tokenize(s)[0])

  0%|          | 0/5541 [00:00<?, ?it/s]

In [60]:
from nltk.tag import SequentialBackoffTagger

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([n.lower() for n in full_n])
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
             return 'NNP'
        else:
             return None

In [61]:
nt = NamesTagger()
print(nt.tag(['Екатерина'])) 
print(nt.tag(['Александр Рогачев'])) 

[('Екатерина', 'NNP')]
[('Александр Рогачев', 'NNP')]


# NER



## Natasha

In [62]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 133 kB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 41 kB 138 kB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26120 sha256=b6b8de5949f5215e04df3589c5a8bd8bfc995b1a88f3896f70e7f3b6e3687c04
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [63]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [67]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = text_corpus[1]
doc = Doc(text)

In [68]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
sent = doc.sents[0]
sent.morph.print()

                   В ADP
          Курганской ADJ|Case=Loc|Degree=Pos|Gender=Fem|Number=Sing
             области NOUN|Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing
                  17 NUM
         сотрудников NOUN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Plur
               ГИБДД PROPN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
          привлечены VERB|Aspect=Perf|Number=Plur|Tense=Past|Variant=Short|VerbForm=Part|Voice=Pass
                   к ADP
      дисциплинарной ADJ|Case=Dat|Degree=Pos|Gender=Fem|Number=Sing
     ответственности NOUN|Animacy=Inan|Case=Dat|Gender=Fem|Number=Sing
                   . PUNCT


In [69]:
sent.syntax.print()

      ┌──► В               case
      │ ┌► Курганской      amod
  ┌──►└─└─ области         obl
  │     ┌► 17              nummod
  │ ┌►┌─└─ сотрудников     nsubj:pass
  │ │ └──► ГИБДД           nmod
┌─└─└───── привлечены      
│ │   ┌──► к               case
│ │   │ ┌► дисциплинарной  amod
│ └──►└─└─ ответственности obl
└────────► .               punct
